In [2]:
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

# alt splits

In [3]:
# import pandas as pd
# alt_splits = pd.read_parquet('/Users/marcinsawinski/Documents/GitHub/factue-task2/data/raw/persuasion/alt_splits.parquet')
# alt_splits

# Trail

In [4]:
from pathlib import Path
import pandas as pd
from factue.utils.vars import PROJECT_ROOT
import numpy as np

base_dir = PROJECT_ROOT / "data/raw/persuasion/trial_data/TRIAL-ANNOTATED"
langs = ["PL", "RU", "BG", "SI"]

records = []

for lang in langs:
    lang_path = base_dir / lang
    raw_docs_path = lang_path / "raw-documents"
    subtask1_path = lang_path / "subtask-1-annotations.txt"
    subtask2_path = lang_path / "subtask-2-annotations.txt"

    # Load all raw documents
    raw_docs = {
        f.name: f.read_text(encoding="utf-8") for f in raw_docs_path.glob("*.txt")
    }

    # Load subtask-2 annotations into a dict
    subtask2_map = {}
    if subtask2_path.exists():
        with subtask2_path.open(encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) >= 4:
                    fname, start, end, *labels = parts
                    key = (fname, int(start), int(end))
                    subtask2_map[key] = labels

    # Parse subtask-1 annotations and extract spans
    with subtask1_path.open(encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 4:
                continue
            fname, start, end, label_bin = parts
            start, end = int(start), int(end)
            text = raw_docs.get(fname, "")[start:end]
            labels_multi = subtask2_map.get((fname, start, end), [])

            records.append(
                {
                    "filename": fname,
                    "start": start,
                    "end": end,
                    "text_lang": lang,
                    "text": text,
                    "label_bin": label_bin.lower() == "true",
                    "labels_multi": labels_multi,
                }
            )

# Create a DataFrame
trial = pd.DataFrame(records)
trial['base_split'] = 'trial'


# Train

In [5]:
from pathlib import Path
import pandas as pd
from factue.utils.vars import PROJECT_ROOT

base_dir = PROJECT_ROOT / "data/raw/persuasion/train_data"
langs = ["PL", "RU", "BG", "SI"]

records = []

for lang in langs:
    lang_path = base_dir / lang
    raw_docs_path = lang_path / "raw-documents"
    subtask1_path = lang_path / "subtask-1-annotations.txt"
    subtask2_path = lang_path / "subtask-2-annotations.txt"

    # Load all raw documents
    raw_docs = {
        f.name: f.read_text(encoding="utf-8") for f in raw_docs_path.glob("*.txt")
    }

    # Load subtask-2 annotations into a dict
    subtask2_map = {}
    if subtask2_path.exists():
        with subtask2_path.open(encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) >= 4:
                    fname, start, end, *labels = parts
                    key = (fname, int(start), int(end))
                    subtask2_map[key] = labels

    # Parse subtask-1 annotations and extract spans
    with subtask1_path.open(encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 4:
                continue
            fname, start, end, label_bin = parts
            start, end = int(start), int(end)
            text = raw_docs.get(fname, "")[start:end]
            labels_multi = subtask2_map.get((fname, start, end), [])

            records.append(
                {
                    "filename": fname,
                    "start": start,
                    "end": end,
                    "text_lang": lang,
                    "text": text,
                    "label_bin": label_bin.lower() == "true",
                    "labels_multi": labels_multi,
                }
            )

# Create a DataFrame
train = pd.DataFrame(records)
train['base_split'] = 'train'


In [6]:
train

,filename,start,end,text_lang,text,label_bin,labels_multi,base_split
0,pl_current_affairs_09_01_2025_n03.txt,0,87,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[],train
1,pl_current_affairs_09_01_2025_n03.txt,89,261,PL,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...,True,"[Questioning_the_Reputation, Name_Calling-Labe...",train
2,pl_current_affairs_09_01_2025_n03.txt,263,872,PL,"(Poseł Władysław Dajczak: Ha, ha, ha! To prawd...",True,"[Conversation_Killer, Appeal_to_Hypocrisy, Loa...",train
3,pl_current_affairs_09_01_2025_n03.txt,874,1322,PL,Kolejne pytanie. Na stronie Wód Polskich pojaw...,True,[Repetition],train
4,pl_current_affairs_09_01_2025_n03.txt,1324,1465,PL,Jeszcze tylko jedna rzecz. Nawet politycy Lewi...,True,[Appeal_to_Popularity],train
...,...,...,...,...,...,...,...,...
994,SI_interpelation_interior_7.txt,0,274,SI,"Zanimivo ob tem dejstvu je, da se pa na jugovz...",True,"[Causal_Oversimplification, Appeal_to_Fear-Pre...",train
995,SI_interpelation_interior_7.txt,276,837,SI,Prav tako poročila obeh policijskih postaj gov...,True,"[Appeal_to_Values, Red_Herring, Doubt]",train
996,SI_interpelation_interior_7.txt,839,2596,SI,Naslednji primer: žena pomočnika generalnega d...,True,"[Causal_Oversimplification, Exaggeration-Minim...",train
997,SI_interpelation_interior_7.txt,2598,5288,SI,"Policisti prav tako dobijo navodila, naj ne sp...",True,"[Consequential_Oversimplification, Exaggeratio...",train


In [7]:
df = pd.concat([trial,train], axis=0)
df


,filename,start,end,text_lang,text,label_bin,labels_multi,base_split
0,pl_abortion_11_04_2024_n01.txt,0,266,PL,Wicemarszałek Włodzimierz Czarzasty:\nWznawiam...,False,[],trial
1,pl_abortion_11_04_2024_n01.txt,268,371,PL,"Panowie, halo, szanujmy wszystkich gości.\nDzi...",False,[],trial
2,pl_abortion_11_04_2024_n01.txt,373,911,PL,"Przystępujemy do rozpatrzenia punktów 10.,\n11...",False,[],trial
3,pl_abortion_11_04_2024_n01.txt,913,1030,PL,Bardzo proszę panią poseł Annę Marię Żukowską\...,False,[],trial
4,pl_abortion_11_04_2024_n01.txt,1032,1828,PL,Panie Marszałku! Wysoka Izbo! Dlaczego kobiety...,True,"[Conversation_Killer, Appeal_to_Pity, Appeal_t...",trial
...,...,...,...,...,...,...,...,...
994,SI_interpelation_interior_7.txt,0,274,SI,"Zanimivo ob tem dejstvu je, da se pa na jugovz...",True,"[Causal_Oversimplification, Appeal_to_Fear-Pre...",train
995,SI_interpelation_interior_7.txt,276,837,SI,Prav tako poročila obeh policijskih postaj gov...,True,"[Appeal_to_Values, Red_Herring, Doubt]",train
996,SI_interpelation_interior_7.txt,839,2596,SI,Naslednji primer: žena pomočnika generalnega d...,True,"[Causal_Oversimplification, Exaggeration-Minim...",train
997,SI_interpelation_interior_7.txt,2598,5288,SI,"Policisti prav tako dobijo navodila, naj ne sp...",True,"[Consequential_Oversimplification, Exaggeratio...",train


In [8]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Default alt_split is same as base_split
df['alt_split'] = df['base_split']

# Select only the 'train' rows
train_rows = df[df['base_split'] == 'train']

# Stratified split: 70% stay as 'train', 30% go to 'dev'
train_idx, dev_idx = train_test_split(
    train_rows.index,
    test_size=0.3,
    stratify=train_rows['text_lang'],
    random_state=42
)

# Assign alt_split values
df.loc[train_idx, 'alt_split'] = 'train'
df.loc[dev_idx, 'alt_split'] = 'dev'

# Move all 'trial' rows to 'dev'
df.loc[df['base_split'] == 'trial', 'alt_split'] = 'dev'

In [9]:
# Step 1: Explode the list of labels
df_exploded = df.explode('labels_multi')

# Step 2: Group by base_split and label, then count
label_counts = df_exploded.groupby(['labels_multi', 'alt_split']).size().unstack(fill_value=0)
label_counts

alt_split,dev,train
labels_multi,,
Appeal_to_Authority,12,52
Appeal_to_Fear-Prejudice,34,67
Appeal_to_Hypocrisy,23,53
Appeal_to_Pity,20,29
Appeal_to_Popularity,28,36
Appeal_to_Time,11,18
Appeal_to_Values,60,84
Causal_Oversimplification,19,42
Consequential_Oversimplification,13,30


In [10]:
df.groupby(["base_split", "alt_split"]).agg('count')

filename  start  end  text_lang  text  label_bin  \
base_split alt_split                                                     
train      dev             300    300  300        300   300        300   
           train           699    699  699        699   699        699   
trial      dev             129    129  129        129   129        129   

                      labels_multi  
base_split alt_split                
train      dev                 300  
           train               699  
trial      dev                 129

# merging 

In [11]:
df.merge(alt_splits, on=['filename','start','end'], how='outer')

NameError: name 'alt_splits' is not defined

In [ ]:
from pathlib import Path

batch_size = 10
output_base = PROJECT_ROOT / "data/preprocessed/persuasion"
output_base.mkdir(parents=True, exist_ok=True)

for split in df["alt_split"].unique():
    for lang in df["text_lang"].unique():
        # Filter by both split and lang
        df_lang_split = df[(df["alt_split"] == split) & (df["text_lang"] == lang)] \
            .sample(frac=1, random_state=42) \
            .reset_index(drop=True)
        if df_lang_split.empty:
            continue  # Skip if no data for this combo

        # Create output directory
        lang_dir = output_base / f"alt_{split}" / lang.lower()
        lang_dir.mkdir(parents=True, exist_ok=True)

        # Save in batches
        for i in range(0, len(df_lang_split), batch_size):
            batch_df = df_lang_split.iloc[i : i + batch_size]
            batch_index = i // batch_size
            batch_filename = f"alt_{split}-{lang.lower()}-part_{batch_index:04d}.parquet"
            batch_path = lang_dir / batch_filename
            batch_df.to_parquet(batch_path, index=False)
            print(f"Saved {batch_path}")

Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0000.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0001.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0002.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0003.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0004.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0005.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-part_0006.parquet
Saved /Users/marcinsawinski/Documents/GitHub/factue-task2/data/preprocessed/persuasion/alt_dev/pl/alt_dev-pl-pa

In [ ]:

from pathlib import Path

batch_size = 20
output_base = PROJECT_ROOT / "data/preprocessed/persuasion/train"
output_base.mkdir(parents=True, exist_ok=True)

for lang in df["text_lang"].unique():
    df_lang = df[df["text_lang"] == lang].reset_index(drop=True)
    lang_dir = output_base / f"train-{lang.lower()}"
    lang_dir.mkdir(parents=True, exist_ok=True)

    # Save in batches of 20
    for i in range(0, len(df_lang), batch_size):
        batch_df = df_lang.iloc[i : i + batch_size]
        batch_index = i // batch_size
        batch_filename = f"batch_{batch_index:04d}.parquet"
        batch_path = lang_dir / batch_filename
        batch_df.to_parquet(batch_path, index=False)
        print(f"Saved {batch_path}")

In [ ]:
df.value_counts("text_lang")

text_lang
BG    363
PL    289
RU    239
SI    108
Name: count, dtype: int64

In [ ]:
from collections import Counter

# Flatten all labels_multi lists into one list
all_labels = df["labels_multi"].explode()

# Drop None or NaN values (in case some entries have empty label lists)
all_labels = all_labels.dropna()

# Count occurrences
label_counts = Counter(all_labels)

# Convert to DataFrame for display or saving
label_counts_df = pd.DataFrame(
    label_counts.items(), columns=["label", "count"]
).sort_values("count", ascending=False)

print(label_counts_df)

                               label  count
4                    Loaded_Language    171
0         Questioning_the_Reputation    138
8                              Doubt    136
14                  Appeal_to_Values    121
1              Name_Calling-Labeling    101
10         Exaggeration-Minimisation     99
5                         Repetition     95
18          Appeal_to_Fear-Prejudice     93
3                Appeal_to_Hypocrisy     73
2                Conversation_Killer     65
20               Appeal_to_Authority     64
6               Appeal_to_Popularity     58
19         Causal_Oversimplification     58
7                        Red_Herring     49
9                        Flag_Waving     43
11  Consequential_Oversimplification     40
12                    Appeal_to_Pity     39
15   Obfuscation-Vagueness-Confusion     38
21           False_Dilemma-No_Choice     30
16                         Straw_Man     29
17                           Slogans     28
22                      Whatabou

In [ ]:
import yaml
from langchain.prompts import PromptTemplate



with open("prompts.yaml", "r") as f:
    prompt_defs = yaml.safe_load(f)

prompt_defs


In [ ]:
from pathlib import Path


In [ ]:
input_dir = Path("data/preprocessed/persuasion")

In [ ]:
for x in input_dir.glob("**/*.parquet"):
    print(x)

In [ ]:
for x in input_dir.glob("**/train/train-*/batch*.parquet"):
    print(x)

# stats

In [ ]:
from pathlib import Path
import pandas as pd
from factue.utils.vars import PROJECT_ROOT

base_dir = PROJECT_ROOT / "data/raw/persuasion/trial_data/TRIAL-ANNOTATED"
langs = ["PL", "RU", "BG", "SI"]

records = []

for lang in langs:
    lang_path = base_dir / lang
    raw_docs_path = lang_path / "raw-documents"
    subtask1_path = lang_path / "subtask-1-annotations.txt"
    subtask2_path = lang_path / "subtask-2-annotations.txt"

    # Load all raw documents
    raw_docs = {
        f.name: f.read_text(encoding="utf-8") for f in raw_docs_path.glob("*.txt")
    }

    # Load subtask-2 annotations into a dict
    subtask2_map = {}
    if subtask2_path.exists():
        with subtask2_path.open(encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) >= 4:
                    fname, start, end, *labels = parts
                    key = (fname, int(start), int(end))
                    subtask2_map[key] = labels

    # Parse subtask-1 annotations and extract spans
    with subtask1_path.open(encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 4:
                continue
            fname, start, end, label_bin = parts
            start, end = int(start), int(end)
            text = raw_docs.get(fname, "")[start:end]
            labels_multi = subtask2_map.get((fname, start, end), [])

            records.append(
                {
                    "filename": fname,
                    "start": start,
                    "end": end,
                    "text_lang": lang,
                    "text": text,
                    "label_bin": label_bin.lower() == "true",
                    "labels_multi": labels_multi,
                }
            )

# Create a DataFrame
trial = pd.DataFrame(records)
trial

,filename,start,end,text_lang,text,label_bin,labels_multi
0,pl_abortion_11_04_2024_n01.txt,0,266,PL,Wicemarszałek Włodzimierz Czarzasty:\nWznawiam...,False,[]
1,pl_abortion_11_04_2024_n01.txt,268,371,PL,"Panowie, halo, szanujmy wszystkich gości.\nDzi...",False,[]
2,pl_abortion_11_04_2024_n01.txt,373,911,PL,"Przystępujemy do rozpatrzenia punktów 10.,\n11...",False,[]
3,pl_abortion_11_04_2024_n01.txt,913,1030,PL,Bardzo proszę panią poseł Annę Marię Żukowską\...,False,[]
4,pl_abortion_11_04_2024_n01.txt,1032,1828,PL,Panie Marszałku! Wysoka Izbo! Dlaczego kobiety...,True,"[Conversation_Killer, Appeal_to_Pity, Appeal_t..."
...,...,...,...,...,...,...,...
124,SI_sample_1.txt,89,423,SI,Zakon o položaju študentov skuša izboljšati ra...,True,"[Consequential_Oversimplification, Appeal_to_V..."
125,SI_sample_1.txt,425,690,SI,"Zakon izboljšuje štipendijsko politiko, kar om...",True,"[Consequential_Oversimplification, Loaded_Lang..."
126,SI_sample_1.txt,692,1133,SI,"Država že sedaj, sicer po dolgem času, namenja...",True,"[Flag_Waving, Appeal_to_Values, Red_Herring]"
127,SI_sample_1.txt,1135,1331,SI,Ob vprašanju digitalne študentske izkaznice sm...,True,[Appeal_to_Fear-Prejudice]


In [ ]:
from pathlib import Path
import pandas as pd
from factue.utils.vars import PROJECT_ROOT

base_dir = PROJECT_ROOT / "data/raw/persuasion/train_data"
langs = ["PL", "RU", "BG", "SI"]

records = []

for lang in langs:
    lang_path = base_dir / lang
    raw_docs_path = lang_path / "raw-documents"
    subtask1_path = lang_path / "subtask-1-annotations.txt"
    subtask2_path = lang_path / "subtask-2-annotations.txt"

    # Load all raw documents
    raw_docs = {
        f.name: f.read_text(encoding="utf-8") for f in raw_docs_path.glob("*.txt")
    }

    # Load subtask-2 annotations into a dict
    subtask2_map = {}
    if subtask2_path.exists():
        with subtask2_path.open(encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) >= 4:
                    fname, start, end, *labels = parts
                    key = (fname, int(start), int(end))
                    subtask2_map[key] = labels

    # Parse subtask-1 annotations and extract spans
    with subtask1_path.open(encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) != 4:
                continue
            fname, start, end, label_bin = parts
            start, end = int(start), int(end)
            text = raw_docs.get(fname, "")[start:end]
            labels_multi = subtask2_map.get((fname, start, end), [])

            records.append(
                {
                    "filename": fname,
                    "start": start,
                    "end": end,
                    "text_lang": lang,
                    "text": text,
                    "label_bin": label_bin.lower() == "true",
                    "labels_multi": labels_multi,
                }
            )

# Create a DataFrame
train = pd.DataFrame(records)
train

,filename,start,end,text_lang,text,label_bin,labels_multi
0,pl_current_affairs_09_01_2025_n03.txt,0,87,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[]
1,pl_current_affairs_09_01_2025_n03.txt,89,261,PL,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...,True,"[Questioning_the_Reputation, Name_Calling-Labe..."
2,pl_current_affairs_09_01_2025_n03.txt,263,872,PL,"(Poseł Władysław Dajczak: Ha, ha, ha! To prawd...",True,"[Conversation_Killer, Appeal_to_Hypocrisy, Loa..."
3,pl_current_affairs_09_01_2025_n03.txt,874,1322,PL,Kolejne pytanie. Na stronie Wód Polskich pojaw...,True,[Repetition]
4,pl_current_affairs_09_01_2025_n03.txt,1324,1465,PL,Jeszcze tylko jedna rzecz. Nawet politycy Lewi...,True,[Appeal_to_Popularity]
...,...,...,...,...,...,...,...
994,SI_interpelation_interior_7.txt,0,274,SI,"Zanimivo ob tem dejstvu je, da se pa na jugovz...",True,"[Causal_Oversimplification, Appeal_to_Fear-Pre..."
995,SI_interpelation_interior_7.txt,276,837,SI,Prav tako poročila obeh policijskih postaj gov...,True,"[Appeal_to_Values, Red_Herring, Doubt]"
996,SI_interpelation_interior_7.txt,839,2596,SI,Naslednji primer: žena pomočnika generalnega d...,True,"[Causal_Oversimplification, Exaggeration-Minim..."
997,SI_interpelation_interior_7.txt,2598,5288,SI,"Policisti prav tako dobijo navodila, naj ne sp...",True,"[Consequential_Oversimplification, Exaggeratio..."


In [ ]:
train.columns

Index(['filename', 'start', 'end', 'text_lang', 'text', 'label_bin',
       'labels_multi'],
      dtype='object')